# Read the tables, we have 1 per model 
# Separate the first n line and the next n and so for each num of labeled samples 
# combine
# write to csv

In [25]:
import pandas as pd 

#model_list = ['xgb', 'mlp', 'dae', 'vime', 'scarf', 'subtab', 'switchtab']
model_list = ['dae', 'vime', 'scarf']
num_samples_list = ['100', '1K', '10K', '20K']
tasks =[#'dashClient_trace.txt_newBitRate_bps', 
                      'vrFragment_trace.txt_vr_frag_thput_mbps', 'vrFragment_trace.txt_vr_burst_thput_mbps',
                      'vrFragment_trace.txt_vr_frag_time', 'vrFragment_trace.txt_vr_burst_time', 
                      #'httpClientRtt_trace.txt_page_load_time',
                      'delay_trace.txt_ul_delay', 'delay_trace.txt_dl_delay']

results_prefix_str = 's3l_unfrozen_'
results = {}
# Read all the results
for model in model_list:
    results_ff = pd.read_csv(results_prefix_str+model+'_test_results.csv', delimiter=",", header=None)
    #print(results_ff)
    results[model] = {}
    results[model]['100'] = results_ff.iloc[0:6]
    results[model]['1K'] = results_ff.iloc[6:12]
    results[model]['10K'] = results_ff.iloc[12:18]
    results[model]['20K'] = results_ff.iloc[18:24]
    
# For each error metric type 
# for each num of samples 
for num_samp in num_samples_list:
    # create 3 err metric dataframnes 
    err_r2 = pd.DataFrame(index=tasks, columns=model_list)
    err_mape = pd.DataFrame(index=tasks, columns=model_list)
    err_mae = pd.DataFrame(index=tasks, columns=model_list)
    err_r2 = err_r2.astype(float)
    err_mape = err_mape.astype(float)
    err_mae = err_mae.astype(float)
    for model in model_list:
        # add one column at a time 
        err_mae.loc[:, model] = results[model][num_samp].iloc[:,1]
        err_mape.loc[:, model] = results[model][num_samp].iloc[:2]
        err_r2.loc[:, model] = results[model][num_samp].iloc[:,3]
    print('Num of samples: ', num_samp)
    #print(err_mae)
    #print(err_mape)
    err_r2 = err_r2.round(3)
    err_mae = err_mae.round(3)
    err_mape = err_mape.round(3)
    #print(err_r2)
    
    # Convert DataFrame to a LaTeX-friendly format
    latex_table = err_r2.apply(lambda row: ' & '.join(row.astype(str)), axis=1)
    latex_table = latex_table + ' \\\\'
    
    # Join the rows with newline characters
    latex_table_str = '\n'.join(latex_table)
    
    print(latex_table_str)
    
# Print with & as the delimiter to make it each to import into latex 

Num of samples:  100
-0.147 & -0.935 & -0.542 \\
0.155 & 0.387 & 0.409 \\
0.224 & -0.067 & 0.706 \\
0.267 & -0.059 & 0.684 \\
0.019 & -0.095 & 0.06 \\
0.074 & 0.152 & 0.395 \\
Num of samples:  1K
0.465 & -0.006 & 0.372 \\
0.604 & 0.645 & 0.622 \\
0.761 & 0.818 & 0.765 \\
0.76 & 0.797 & 0.766 \\
0.053 & -0.049 & 0.141 \\
0.566 & 0.623 & 0.656 \\
Num of samples:  10K
0.247 & -0.615 & 0.716 \\
0.713 & 0.734 & 0.742 \\
0.825 & 0.833 & 0.804 \\
0.81 & 0.839 & 0.825 \\
0.214 & 0.243 & 0.232 \\
0.725 & 0.714 & 0.713 \\
Num of samples:  20K
0.675 & 0.561 & 0.741 \\
0.752 & 0.765 & 0.739 \\
0.835 & 0.846 & 0.832 \\
0.83 & 0.846 & 0.823 \\
0.141 & 0.233 & 0.234 \\
0.736 & 0.751 & 0.706 \\
